<a href="https://colab.research.google.com/github/quentin12349876/hello-world/blob/main/solver/Base_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import matplotlib.animation

#Interpolation of data values form csv file, only to be used if interpolation is required
path = "initial_conditions.csv"
df = pd.read_csv(path, encoding='latin1') #read the csv file
array_data = df.to_numpy()  # converting data to numpy array
df.columns = (df.columns            # Access all column names in the DataFrame
       .str.strip()       # Remove leading and trailing whitespace from each column name
       .str.replace('\ufeff', ''))# Remove the UTF-8 BOM (Byte Order Mark) character if present

df = df.rename(columns={  #reanme the title of each columns
    df.columns[0]: 'concentration',
    df.columns[1]: 'distance'})
# converting "concentration" into numerical values
df['concentration'] = pd.to_numeric(df['concentration'], errors='coerce')
# converting "distance" into numerical values
df['distance'] = pd.to_numeric(df['distance'], errors='coerce')
df = df.dropna() # Remove rows that contain NaN values in any column
df = df.sort_values(by='concentration') # Sort the data vaule in  concentration

# Remove duplicate concentration values
# Keeps the first occurrence to ensure unique x-values for interpolation
df = df.drop_duplicates(subset='concentration')
x = df['concentration'].to_numpy() #converitng to numpy
y = df['distance'].to_numpy() #converting to numpy

# Create a linear interpolation function based on the original data
# bounds_error=False allows extrapolation outside the original range
f = interp1d(x, y, kind='linear', bounds_error=False)
num_between = 5

# Create a new concentration array with evenly spaced points
# (len(x)-1) intervals, with 'num_between' points added per interval
x_new = np.linspace(x.min(), x.max(), (len(x) - 1) * num_between + 1)

# Compute interpolated distance values for the new concentration points
y_new = f(x_new)

# Create a new DataFrame containing the extended (interpolated) data
extended_df = pd.DataFrame({
    'concentration': x_new,
    'distance': y_new})

# Save the extended dataset to a CSV file
extended_df.to_csv("extended_data.csv", index=False)


class AdvectionSolver:
  #Finite difference solver for the 1D advection equation: d(theta)/dt + U*d(theta)/dx = 0

    def __init__(self, L, dx, dt, T, U):
        self.L = L
        self.dx = dx
        self.dt = dt
        self.T = T
        self.U = U

        # Spatial grid
        self.x_new = x_new
        self.num_points = len(self.x_new)
        self.num_time_steps = int(self.T / self.dt)

        if self.U <= 0:
            raise ValueError("This solver assumes flow velocity U > 0.")

    def _solve_implicit(self, theta_initial, boundary_theta):
        """
        Implicit Upwind Scheme (Standard BTBS).
        Unconditionally stable.
        Equation: theta_i^{n+1} = (theta_i^n + C * theta_{i-1}^{n+1}) / (1 + C)
        """
        print(f"Using **Implicit Scheme (Standard BTBS)**.")

        theta = y_new
        N = self.num_points

        theta_history = np.zeros((self.num_time_steps + 1, N))
        theta_history[0, :] = theta

        C = abs(self.U) * self.dt / self.dx # Parameter C for the implicit formula
        divisor = 1.0 + C

        for n in range(self.num_time_steps):
            theta_next = np.zeros_like(theta)

            # Boundary Condition
            theta_next[0] = boundary_theta

            # Forward Substitution
            for i in range(1, N):
                numerator = theta[i] + C * theta_next[i-1]
                theta_next[i] = numerator / divisor

            theta = theta_next
            theta_history[n+1, :] = theta

        return theta_history

    def compute_solution(self, theta_initial, boundary_theta):

        return self._solve_implicit(theta_initial, boundary_theta)

#define variables based on conditions specified
L =
T =
dx =
dt =
U =
boundary_theta =


solver_A = AdvectionSolver(L, dx, dt, T, U)

#defenition of theta_initial when is theta is a constant at t = 0
theta_initial = np.zeros_like(solver_A.x_new)
theta_initial[0] = boundary_theta

#defenition theta_initial when theta has to be interpolated
theta_initial = f(solver_A.x_new)


res_A = solver_A.compute_solution(theta_initial, boundary_theta) #compute solution to equation

fig, ax = plt.subplots(figsize=(4,3)) # creating a figure to plot graph on

plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150
plt.ioff() # interactive off

def animate(t):
  plt.cla()
  y = res_A[t]
  plt.plot(x_new, y)
  plt.xlim(0,L)
  plt.ylim(0, 300)
  ax.set_title("Concentration of pollutant at t = " + str(dt*t) + "s", fontsize=10, verticalalignment='top')
  plt.xlabel("x", fontsize=8)
  plt.ylabel("Concentration", fontsize=8)

anim = matplotlib.animation.FuncAnimation(fig, animate, frames=len(res_A)) #animatng graph to show the change in concentration over time
anim

SyntaxError: invalid syntax (ipython-input-716656768.py, line 108)